<a href="https://colab.research.google.com/github/ahlqui/VeloxChemColabs/blob/main/MolecularDynamicsTwoMolecules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Generate two force field from XYZ and run dynamics with two non-bonded molecules


This notebook requires that you have a molecular structure. Two options:

Define Molecule as XYZ-coordinates: Can be generatred with free software such as www.avogadro.cc

Define Molecule with SMILES code, A molecule can be defined using a SMILES code (example below). We have two suggested ways to generate smiles from structure. 1) Sketch your molecule at https://www.rcsb.org/chemical-sketch and the SMILES code will be shown right below the structure. 2) Build your molecule at https://molview.org/ , go to Tools/Information Card and it will show you the SMILES code. Just copy/paste it into the SMILES box below.

Example of xyz-coordinates for CO2:

O 0.00 0.00 0.00

C 0.00 0.00 1.20

O 0.00 0.00 2.40

Example of SMILES for CO2:

O=C=O

In [1]:
#@title Install VeloxChem and dependencies (ca 2 min)
#@markdown Before you can run anything a few things need to be installed. It all runs on the Google Colab server an you do not need to install anything on your computer.
%%capture
import sys

! wget -qnc https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh
! bash Miniforge3-Linux-x86_64.sh -bfp /usr/local
! mamba config --set auto_update_conda false
import sys
sys.path.append('/usr/local/lib/python3.11/site-packages/')
!mamba install -y -c conda-forge -c veloxchem veloxchem xtb-python py3Dmol MDAnalysis openmm matplotlib python=3.11
! pip install rdkit
import veloxchem as vlx

In [2]:
xyz_coordinates1 = """
O          0.25680        2.99920       -0.40240
C         -0.93050        3.54070       -0.97090
C         -2.12350        3.46520        0.00440
O         -2.98150        2.36130       -0.26840
H         -1.18040        3.04170       -1.90750
H         -0.74380        4.58820       -1.21180
H         -1.79680        3.48500        1.04560
H         -2.73330        4.35750       -0.13780
C          3.99970        0.60540       -0.26610
O          2.79590        1.12030       -0.81910
O          3.46290       -1.74950       -0.00830
C          4.24210       -0.83130       -0.75850
H          3.97320        0.64730        0.82370
H          4.83770        1.22260       -0.59360
H          4.02160       -0.90970       -1.82410
H          5.29640       -1.07840       -0.62600
C         -3.37800       -2.42880       -0.26530
O         -2.76530       -1.26260        0.25790
C         -2.59290       -3.67410        0.18220
H         -4.39910       -2.47900        0.11470
H         -3.43720       -2.39590       -1.35440
H         -2.23790       -3.56190        1.20730
H         -3.26230       -4.53520        0.16240
C          1.35680        3.03650       -1.30510
H          1.14600        2.47070       -2.21250
H          1.51250        4.07260       -1.60850
C          2.65040        2.52590       -0.64580
H          2.68440        2.80510        0.40820
H          3.49500        3.01720       -1.13090
C          3.67850       -3.10750       -0.36850
H          4.70790       -3.37030       -0.12130
H          3.55330       -3.25870       -1.44200
C          2.71730       -4.01900        0.41480
H          3.14350       -5.02050        0.48580
H          2.59080       -3.65030        1.43410
C         -3.30870       -0.05010       -0.24850
H         -3.26530       -0.05230       -1.33850
H         -4.35640        0.04060        0.04030
C         -2.50470        1.14410        0.28640
H         -1.46370        1.02610        0.01280
H         -2.55230        1.18120        1.37550
O          1.47390       -4.11280       -0.26870
C          0.45950       -4.77610        0.47460
H          0.85440       -5.72290        0.84610
H          0.16360       -4.18780        1.34370
C         -0.74340       -5.07390       -0.43880
H         -1.36820       -5.85360       -0.00020
H         -0.37910       -5.47770       -1.38380
O         -1.51910       -3.91440       -0.72000

"""

In [3]:
#@title Define your molecule, either asa SMILES code or xyz-coordinates above
#@markdown - Enter the SMILES code
smiles_code1 = 'O/C=C\\C=O' #@param {type:"string"}
#@markdown - If you want to use the xyz coordinates in the previous cell check box
use_xyz = True # @param {type:"boolean"}
if use_xyz == False:
  molecule1 = vlx.Molecule.read_smiles(smiles_code1)
else:
  molecule1 = vlx.Molecule.read_str(xyz_coordinates1)
print('Structure of the molecule entered: ')
molecule1.show(atom_indices=True)
#@markdown - Give your Gromacs files a name
gromacs_file1 = 'mol1' #@param {type:"string"}
#@markdown - Give your molecule a residue name (three letters)
residue_name1 = 'M1' #@param {type:"string"}


Structure of the molecule entered: 


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [4]:
#@title Generate a force field and write Gromacs files for the first molecule
#@markdown - Example timing: 21-Crown-7 (2 min)
ff_gen = vlx.MMForceFieldGenerator()
basis = vlx.MolecularBasis.read(molecule1, 'sto-3g')
ff_gen.ostream.mute()
ff_gen.create_topology(molecule1, basis)

ff_gen.write_gromacs_files(gromacs_file1, residue_name1)

In [5]:
xyz_coordinates2 = """
N     0.4785967928750  -0.4330741880750  -0.4934317502006
H     1.3794564278886   0.0429751463168  -0.5120458348956
C     0.4573285404340  -1.3154740355836  -1.6452845209868
H     1.2794157350200  -2.0270971721773  -1.5868316520123
H    -0.4697908768414  -1.8882598290096  -1.6558792536570
C     0.3747047426388  -1.2102491878280   0.7331072010194
H    -0.4082292852521  -1.9567463178277   0.6148319854093
H     1.2952953868059  -1.7540437197433   0.9183116804327
C     0.6956425554192   0.9059967227241  -5.3588904651214
C     1.6690431285267  -0.0704102447688  -5.0614336089256
C     1.5905279342425  -0.7922292640998  -3.8527793116028
C     0.5421211238334  -0.5430166236398  -2.9451176135833
C    -0.4312572408155   0.4285446612223  -3.2497814984180
C    -0.3564249048118   1.1570067527526  -4.4544417171718
H     0.7549521548685   1.4652126122086  -6.2812972715425
H     2.3338053657174  -1.5390048087124  -3.6163728176857
H    -1.2215586002930   0.6165771400284  -2.5371827271696
C    -0.5809783149118   1.1061887532548   4.2819306994214
C    -1.0539691464599  -0.2152180430193   4.1484967454886
C    -0.7402866542775  -0.9585904738513   2.9917812697427
C     0.0501603450518  -0.3918335577218   1.9715484114033
C     0.5329231859523   0.9234697295847   2.1220017776407
C     0.2096151223583   1.6793509886579   3.2655299818144
H    -0.8298576406810   1.6816374807225   5.1614537759970
H    -1.1155297675244  -1.9639771384900   2.8737258346041
H     1.1311551788061   1.3586475153088   1.3389498331805
C     2.7985412681099  -0.3440921389086  -6.0320361103160
H     3.0137177612643  -1.4119025048439  -6.0784904425001
H     2.5433990952730  -0.0090827377477  -7.0376742707821
H     3.7013243404231   0.1769923076871  -5.7132670703364
C    -1.3956677617432   2.2110713683443  -4.7713360805252
H    -2.2850215726603   2.0829577471538  -4.1532353918110
H    -0.9892606342753   3.2051959925139  -4.5854917335667
H    -1.6989550492144   2.1496453780473  -5.8166145479409
C    -1.9096053739985  -0.8269385079269   5.2372774019485
H    -1.8730938416183  -1.9157634078951   5.1986764605346
H    -1.5629099527925  -0.5125717843390   6.2218512212214
H    -2.9465170241487  -0.5126306418706   5.1188520156529
C     0.7055071531436   3.1039291474455   3.3878074482962
H     0.6996321098764   3.5926125742168   2.4126899821207
H     0.0720511304863   3.6841458846608   4.0587668370105
H     1.7245462284362   3.1151446899290   3.7742478882927
"""


In [6]:
#@title Define your molecule, either asa SMILES code or xyz-coordinates above
#@markdown - Enter the SMILES code
smiles_code2 = 'O/C=C\\C=O' #@param {type:"string"}
#@markdown - If you want to use the xyz coordinates in the previous cell check box
use_xyz = True # @param {type:"boolean"}
if use_xyz == False:
  molecule2 = vlx.Molecule.read_smiles(smiles_code2)
else:
  molecule2 = vlx.Molecule.read_str(xyz_coordinates2)
print('Structure of the molecule entered: ')
molecule2.show(atom_indices=True)
#@markdown - Give your Gromacs files a name
gromacs_file2 = 'mol2' #@param {type:"string"}
#@markdown - Give your molecule a residue name (three letters)
residue_name2 = 'M2' #@param {type:"string"}

Structure of the molecule entered: 


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
#@title Generate a force field and write Gromacs files for the second molecule
#@markdown - Example timing: diaryl-amine (2 min)
ff_gen = vlx.MMForceFieldGenerator()
basis = vlx.MolecularBasis.read(molecule2, 'sto-3g')
ff_gen.ostream.mute()
ff_gen.create_topology(molecule2, basis)

ff_gen.write_gromacs_files(gromacs_file2, residue_name2)

In [ ]:
%%capture
#@title Combine two MM molecules
# Read in the file
with open(gromacs_file1 + '.gro', 'r') as file:
  filedata = file.read()

# Write the file out again
with open(gromacs_file1 + '.gro', 'w') as file:
  file.write(filedata)

# Read in the file
with open(gromacs_file2 + '.gro', 'r') as file:
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('MOL', 'MO2')

# Write the file out again
with open(gromacs_file2 + '.gro', 'w') as file:
  file.write(filedata)

#@title Combine two MM molecules
# Read in the file
with open(gromacs_file2 + '.itp', 'r') as file:
  filedata = file.read()

# Replace the target string
filedata = filedata.replace('MOL', 'MO2')

# Write the file out again
with open(gromacs_file2 + '.itp', 'w') as file:
  file.write(filedata)

# Read in the file
with open(gromacs_file1 + '.top', 'r') as file:
  filedata = file.read()

# Write the file out again
with open('combined.top', 'w') as file:
  file.write(filedata)

with open("combined.top", "r") as in_file:
    buf = in_file.readlines()

with open("combined.top", "w") as out_file:
    for line in buf:
        if line == "#include \"mol1.itp\"\n":
            line = line + "#include \"mol2.itp\"\n"
        if line == "M1\n":
            line = line + "M2\n"
        if line == "M1                1\n":
            line = line + "M2                1\n"
        out_file.write(line)

import MDAnalysis as mda
u1 = mda.Universe(gromacs_file1 + '.gro')
u2 = mda.Universe(gromacs_file2 + '.gro')
u = mda.Merge(u1.atoms, u2.atoms)
with mda.Writer('combined.gro') as gro:
    gro.write(u)


In [ ]:
#@title Run dynamics
#@markdown - Example timings: 200 ps MD of ibuprofen (22 sec)
from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout

gro = GromacsGroFile('combined' + '.gro')
top = GromacsTopFile('combined' + '.top')
system = top.createSystem(nonbondedMethod=NoCutoff)
#@markdown - Set the timestep in femtoseconds
#time_step = 2 #@param {type:"integer"}
time_step = 2 #@param [1, 2]
#@markdown - Set the temperature in Kelvin
temp = 300 #@param {type:"slider", min:200, max:700, step:1}
integrator = LangevinMiddleIntegrator(temp*kelvin, 1/picosecond, 0.001*time_step*picoseconds)
simulation = Simulation(top.topology, system, integrator)
simulation.context.setPositions(gro.positions)
simulation.minimizeEnergy()
#@markdown - Set the number of steps you want to take (ex 100000 steps * 2 fs= 200 ps)
steps = 100000 #@param {type:"integer"}
#@markdown - Number of snapshots saved. Select how many structures you want to save from your dynamics run
snapshots = 50 #@param {type:"integer"}
outputdata = steps / snapshots
simulation.reporters.append(PDBReporter('combined' + '.pdb', outputdata))
simulation.reporters.append(StateDataReporter(stdout, outputdata, step=True,
        potentialEnergy=True, temperature=True))
simulation.step(steps)



In [ ]:
#@title Visualize the dynamics
import MDAnalysis as mda
from MDAnalysis.analysis import align, rms
mobile = mda.Universe('combined' + '.pdb')
ref = mda.Universe('combined' + '.pdb')

mobile.trajectory[-1]  # set mobile trajectory to last frame
ref.trajectory[0]  # set reference trajectory to first frame

aligner = align.AlignTraj(mobile, ref, select='resname MOL', in_memory=True).run()

with mda.Writer('combinedaligned.pdb', mobile.atoms) as w:
    for ts in mobile.trajectory:
        w.write(mobile.atoms)

import py3Dmol as p3d
interval = 200 #@param  {type:"slider", min:100, max:2000, step:100}
print('Output geometry:')
viewer = p3d.view(width=400,height=250)
viewer.addModelsAsFrames(open('combinedaligned' + '.pdb', 'r').read(),'pdb', {'keepH': True})
viewer.animate({'interval': interval, 'loop': "forward", 'reps': 25})
viewer.setStyle({"stick":{},"sphere": {"scale":0.25}})
viewer.zoomTo()
viewer.show()